In [1]:
# IMG_PATH = "../input/animals-data/dataset/"
IMG_PATH = "C:/TCC_ForaDoOneDrive/CNRPark-Patches-150x150/A/Imagens Selecionadas 512x512/"
IMG_HEIGHT = 512  # The images are already resized here
IMG_WIDTH = 512  # The images are already resized here

# redimensiona a imagem
# img = cv2.resize(
# img, (160, 160), interpolation=cv2.INTER_CUBIC)


SEED = 42
TRAIN_RATIO = 0.75
VAL_RATIO = 1 - TRAIN_RATIO
SHUFFLE_BUFFER_SIZE = 100

LEARNING_RATE = 1e-3
EPOCHS = 2
TRAIN_BATCH_SIZE = 32  # Let's see, I don't have GPU, Google Colab is best hope
TEST_BATCH_SIZE = 32  # Let's see, I don't have GPU, Google Colab is best hope
FULL_BATCH_SIZE = 32

AUTOENCODER_MODEL_PATH = "baseline_autoencoder.pt"

# ENCODER_MODEL_PATH = "deep_encoder.pt"
# DECODER_MODEL_PATH = "deep_decoder.pt"

ENCODER_MODEL_PATH = "baseline_encoder.pt"
DECODER_MODEL_PATH = "baseline_decoder.pt"

EMBEDDING_SHAPE = (1, 64, 64, 64)
# TEST_RATIO = 0.2

APP_PATH = "C:/Users/albs_/source/repos/tcc-machine-vision/image_similarity-master/image_similarity/"
EMBEDDING_PATH = APP_PATH + "data_embedding_f.npy"

In [2]:
#from flask import Flask, request, json
import torch_model
#import config
import torch
import numpy as np

from sklearn.neighbors import NearestNeighbors
import torchvision.transforms as T
import os
import cv2
from PIL import Image
from sklearn.decomposition import PCA

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = "cpu"

# Load the model before we start the server
encoder = torch_model.ConvEncoder()

print('ENCODER_MODEL_PATH: ' + ENCODER_MODEL_PATH)

# Load the state dict of encoder
encoder.load_state_dict(torch.load(ENCODER_MODEL_PATH, map_location=device))
encoder.eval()
encoder.to(device)

# Loads the embedding
embedding = np.load(EMBEDDING_PATH)

print("Loaded model and embeddings")


def compute_similar_images(image_tensor, num_images, embedding, device):
    """
    Given an image and number of similar images to generate.
    Returns the num_images closest neares images.

    Args:
    image_tenosr: PIL read image_tensor whose similar images are needed.
    num_images: Number of similar images to find.
    embedding : A (num_images, embedding_dim) Embedding of images learnt from auto-encoder.
    device : "cuda" or "cpu" device.
    """

    print('compute_similar_images')
    
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        image_embedding = encoder(image_tensor).cpu().detach().numpy()

    print(image_embedding.shape)

    flattened_embedding = image_embedding.reshape((image_embedding.shape[0], -1))
    print(flattened_embedding.shape)

    knn = NearestNeighbors(n_neighbors=num_images, metric="cosine")
    knn.fit(embedding)

    _, indices = knn.kneighbors(flattened_embedding)
    indices_list = indices.tolist()
    print(indices_list)
    return indices_list


def compute_similar_features(image, num_images, embedding, nfeatures=30):
    """
    Given a image, it computes features using ORB detector and finds similar images to it
    Args:
    image: Opencv read Image whose features and simlar images are required.
    num_images: Number of similar images required.
    embedding: 2 Dimensional Embedding vector.
    nfeatures: (optional) Number of features ORB needs to compute
    """

    orb = cv2.ORB_create(nfeatures=nfeatures)

    # Detect features
    keypoint_features = orb.detect(image)
    # compute the descriptors with ORB
    keypoint_features, des = orb.compute(image, keypoint_features)

    # des contains the description to features

    des = des / 255.0
    des = np.expand_dims(des, axis=0)
    des = np.reshape(des, (des.shape[0], -1))
    # print(des.shape)
    # print(embedding.shape)

    pca = PCA(n_components=des.shape[-1])
    reduced_embedding = pca.fit_transform(
        embedding,
    )
    # print(reduced_embedding.shape)

    knn = NearestNeighbors(n_neighbors=num_images, metric="cosine")
    knn.fit(reduced_embedding)
    _, indices = knn.kneighbors(des)

    indices_list = indices.tolist()
    # print(indices_list)
    return indices_list




# @app.route("/simfeat", methods=["POST"])
# def simfeat():
#     r = request.files["image"]
#     # print("Hi")
#     # convert string of image data to uint8
#     nparr = np.fromstring(r.data, np.uint8)
#     # decode image
#     img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#     indices_list = compute_similar_features(img, num_images=5, embedding=embedding)
#     # Need to display the images
#     return (
#         json.dumps({"indices_list": indices_list}),
#         200,
#         {"ContentType": "application/json"},
#     )

#print('config.APP_PATH: ' + config.APP_PATH)

image = Image.open("C:/TCC_ForaDoOneDrive\CNRPark-Patches-150x150/A/Search/busy 20150703_0920_14 512x512.jpg")
print(image)
image_tensor = T.ToTensor()(image)
#print(image_tensor)
image_tensor = image_tensor.unsqueeze(0)
#print(image_tensor)
indices_list = compute_similar_images(
    image_tensor, num_images=5, embedding=embedding, device=device
)
print(indices_list)


ENCODER_MODEL_PATH: baseline_encoder.pt


RuntimeError: Error(s) in loading state_dict for ConvEncoder:
	Missing key(s) in state_dict: "conv4.weight", "conv4.bias", "conv5.weight", "conv5.bias". 